In [42]:
import pandas as pd
from funções.amostras import gerar_amostra
from funções.descompactando import descompactando
import os
from openpyxl import Workbook

 A função a seguir aborda várias etapas, incluindo a extração de arquivos RAR, leitura e processamento de arquivos CSV, manipulação de dados, limpeza, consolidação em um único DataFrame e os salvando em comvustiveis.csv

 Você pode encontrar essa função sendo definida em Funções/descompactando.py

In [ ]:
descompactando()

Esse código realiza várias etapas relacionadas à manipulação e análise de dados de combustíveis automotivos em um arquivo CSV. Essencialmente, essa função lê um arquivo CSV contendo dados de combustíveis automotivos, tenta diferentes encodings para leitura, seleciona uma amostra aleatória dos dados e retorna essa amostra como um DataFrame. é possivel fazer essa analise com varias amostras.

Você pode encontrar essa função sendo definida em Funções/amostras.py

In [5]:
amostra_df = gerar_amostra()

In [6]:
amostra_df.head()

,Regiao - Sigla,Estado - Sigla,Municipio,Produto,Valor de Venda,Unidade de Medida,Bandeira
Data da Coleta,,,,,,,
2012-03-20,NE,BA,ITAMARAJU,DIESEL S50,2.029,Litro,RAIZEN
2016-05-24,NE,CE,MARACANAU,GASOLINA,3.970,Litro,SP
2022-06-20,SE,SP,FRANCA,GASOLINA ADITIVADA,7.090,Litro,RAIZEN
2013-11-26,CO,DF,BRASILIA,DIESEL,2.420,Litro,PETROBRAS DISTRIBUIDORA S.A.
2015-03-25,S,PR,TOLEDO,ETANOL,2.239,Litro,BRANCA


Checagem

In [8]:
indices = set(amostra_df.index.tolist())
print(indices)

{'2013-03-21', '2020-04-07', '2016-10-05', '2013-02-11', '2013-09-16', '2019-05-16', '2022-03-04', '2022-01-24', '2021-03-15', '2012-08-09', '2019-09-02', '2020-02-12', '2022-12-26', '2017-04-18', '2020-11-30', '2015-02-02', '2015-04-21', '2016-05-24', '2012-05-22', '2019-10-07', '2021-07-25', '2022-11-08', '2021-06-18', '2016-03-28', '2013-01-17', '2017-02-06', '2017-12-05', '2019-07-04', '2016-11-28', '2020-03-19', '2012-05-21', '2020-12-01', '2012-01-16', '2021-02-24', '2012-05-24', '2021-11-11', '2015-04-09', '2019-07-08', '2015-09-15', '2012-10-10', '2022-09-29', '2018-12-12', '2022-07-27', '2015-08-04', '2020-07-06', '2021-01-18', '2019-04-22', '2012-08-15', '2020-11-18', '2018-01-31', '2021-07-20', '2021-05-14', '2016-01-20', '2012-10-25', '2018-11-16', '2013-09-09', '2016-11-04', '2013-12-04', '2021-08-03', '2016-05-25', '2021-12-24', '2013-07-23', '2012-07-31', '2019-11-06', '2020-04-28', '2019-11-22', '2016-01-26', '2022-08-25', '2018-08-23', '2016-09-27', '2020-12-25', '2012

In [10]:
formato_dia_mes_ano = "%d/%m/%Y"
formato_ano_mes_dia = "%Y-%m-%d"

tem_formato_dia_mes_ano = False
tem_formato_ano_mes_dia = False

for indice in indices:
    try:
        pd.to_datetime(indice, format=formato_dia_mes_ano)
        tem_formato_dia_mes_ano = True
    except ValueError:
        pass

    try:
        pd.to_datetime(indice, format=formato_ano_mes_dia)
        tem_formato_ano_mes_dia = True
    except ValueError:
        pass

if tem_formato_dia_mes_ano and tem_formato_ano_mes_dia:
     print("A lista de índices contém ambos os formatos de datas.(Precisa de Tratamento)")
else:
    print("A lista de índices não contém ambos os formatos de datas.(Tudo OK)")

A lista de índices não contém ambos os formatos de datas.(Tudo OK)


In [11]:
amostra_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1675306 entries, 2012-03-20 to 2014-04-23
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Regiao - Sigla     1675306 non-null  object 
 1   Estado - Sigla     1675306 non-null  object 
 2   Municipio          1675306 non-null  object 
 3   Produto            1675306 non-null  object 
 4   Valor de Venda     1675306 non-null  float64
 5   Unidade de Medida  1675306 non-null  object 
 6   Bandeira           1675306 non-null  object 
dtypes: float64(1), object(6)
memory usage: 102.3+ MB


In [12]:
amostra_df.shape

(1675306, 7)

In [17]:
amostra_df.isnull().sum()

Regiao - Sigla       0
Estado - Sigla       0
Municipio            0
Produto              0
Valor de Venda       0
Unidade de Medida    0
Bandeira             0
dtype: int64

Analise previa das amostras aleatorias

In [24]:
data_minima = amostra_df.index.min()
data_maxima = amostra_df.index.max()

print("Data Mínima:", data_minima)
print("Data Máxima:", data_maxima)

Data Mínima: 2012-01-02
Data Máxima: 2022-12-30


In [31]:
analise_objects = amostra_df.select_dtypes('object').describe().transpose()
analise_objects.columns = ['Linhas', 'Unicos', 'Mais Repetido', 'Frequência']
analise_objects = analise_objects.rename_axis('Coluna')

def format_number_with_commas(number):
    return '{:,.0f}'.format(number).replace(',', '.')

columns_to_format = [col for col in analise_objects.columns if col not in ['Mais Repetido']]
analise_objects[columns_to_format] = analise_objects[columns_to_format].applymap(format_number_with_commas)

analise_objects

,Linhas,Unicos,Mais Repetido,Frequência
Coluna,,,,
Regiao - Sigla,1.675.306,5,SE,820.931
Estado - Sigla,1.675.306,27,SP,471.838
Municipio,1.675.306,615,SAO PAULO,53.810
Produto,1.675.306,7,GASOLINA,513.317
Unidade de Medida,1.675.306,2,Litro,1.645.052
Bandeira,1.675.306,122,BRANCA,546.454


In [37]:
analise_floats = amostra_df.select_dtypes('float').describe().transpose()
analise_floats.columns = ['Linhas', 'Média', 'Variancia', 'Minimo', '25%', '50%', '75%', 'Maximo']
analise_floats = analise_floats.rename_axis('Coluna')

analise_floats

,Linhas,Média,Variancia,Minimo,25%,50%,75%,Maximo
Coluna,,,,,,,,
Valor de Venda,1675306.0,3.417114,1.257199,1.189,2.579,3.09,3.89,9.27


In [46]:
if not os.path.exists("Tabelas"):
    os.makedirs("Tabelas")

amostras_df.to_csv('Tabelas/amostras_df.csv')
analise_objects.to_csv('Tabelas/analise_objects.csv')
analise_floats.to_csv('Tabelas/analise_floats.csv')

NameError: name 'amostras_df' is not defined